In [1]:
!pip install dash
!pip install ipython-sql
!pip install psycopg2-binary
!pip install pandas
!pip install numpy
!pip install numpy matplotlib
!pip install folium
!pip install geojson
!pip install jobspy

In [2]:
import numpy as np
import pandas as pd
import psycopg2
import pandas.io.sql as sqlio
import matplotlib.pyplot as plt
import pylab as pl
import folium
import json
import os
from folium import plugins
%matplotlib inline

import plotly.graph_objects as go
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output, State
import plotly.express as px


<ipython-input-2-9a3040288960>:15: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html
<ipython-input-2-9a3040288960>:16: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc


In [3]:
psql = psycopg2.connect(host='192.168.10.163', port='5432', database='Otodom', user='barto', password='biznes')

In [4]:
cur = psql.cursor()
sql_rent = "select * from Warsaw_rent;"
sql_sell = "select * from Warsaw_sell;"
dat_rent = sqlio.read_sql_query(sql_rent, psql)
dat_sell = sqlio.read_sql_query(sql_sell, psql)
conn = None

In [5]:
# my_map = folium.Map()
# world = folium.Map(
#     zoom_start=11,
#     location=[52.2380549, 21.0293513], prefer_canvas=True)

In [6]:
lok_sell = dat_sell[["latitude","longitude","price"]]
lok_rent = dat_rent[["latitude","longitude","rent_price"]]

In [7]:
a = 0
for index, row in lok_sell.iterrows():
    if np.isnan(lok_sell.at[index,'latitude']) or np.isnan(lok_sell.at[index,'longitude']) or np.isnan(lok_sell.at[index,'price']):
        lok_sell = lok_sell.drop([index])
        a += 1
print("Drop rows where Nan from table sell: ", a)

b = 0
for index, row in lok_rent.iterrows():
    if np.isnan(lok_rent.at[index,'latitude']) or np.isnan(lok_rent.at[index,'longitude']) or np.isnan(lok_rent.at[index,'rent_price']):
        lok_rent = lok_rent.drop([index])
        b += 1
print("Drop rows where Nan from table rent: ", b)

Drop rows where Nan from table sell:  311
Drop rows where Nan from table rent:  182


In [8]:
lok_sell = lok_sell.reset_index(drop=True)
lok_rent = lok_rent.reset_index(drop=True)

In [9]:
# world = plugins.MarkerCluster().add_to(world)

In [10]:
# for row in range(len(lok_rent.index)):
#     folium.CircleMarker(
#         location=[lok_rent.at[row,'latitude'], lok_rent.at[row,'longitude']],
#         radius=3,
#         popup='Rent price: '+ str(lok_rent.at[row,'rent_price']),
#         color='blue',
#         fill=True,
#         fill_color='blue',
#         fill_opacity=1
#     ).add_to(world)

In [11]:
# for row in range(len(lok_sell.index)):
#     folium.CircleMarker(
#         location=[lok_sell.at[row,'latitude'], lok_sell.at[row,'longitude']],
#         radius=3,
#         popup='Sell price: ' + str(lok_sell.at[row,'price']),
#         color='red',
#         fill=True,
#         fill_color='red',
#         fill_opacity=1
#     ).add_to(world)

In [12]:
# def Sell_Price_Limiter(Price_MIN_Sell, Price_MAX_Sell):
#     my_world = world
#     lok_sell_limit = lok_sell[lok_sell.price.between(Price_MIN_Sell, Price_MAX_Sell, inclusive=False)]
#     lok_sell_limit = lok_sell_limit.reset_index(drop=True)
#     for row in range(len(lok_sell_limit.index)):
#         folium.CircleMarker(
#             location=[lok_sell_limit.at[row,'latitude'], lok_sell_limit.at[row,'longitude']],
#             radius=3,
#             popup='Sell price: ' + str(lok_sell_limit.at[row,'price']),
#             color='red',
#             fill=True,
#             fill_color='red',
#             fill_opacity=1
#         ).add_to(my_world)
#     html_string = my_world.get_root().render()

In [13]:
# world.save('Final_visualization.html')
# world
# html_string = world.get_root().render()
#  open('Final_visualization.html', 'r').read(

In [14]:
# world = folium.Map(
#     zoom_start=11,
#     location=[52.2380549, 21.0293513], prefer_canvas=True)
# world = plugins.MarkerCluster().add_to(world)
# html_string = my_world.get_root().render()

In [15]:
def world_new():    
    my_world = folium.Map(
    zoom_start=11,
    location=[52.2380549, 21.0293513], prefer_canvas=True)
    my_world = plugins.MarkerCluster().add_to(my_world)
    return my_world

In [16]:
def start():
    start = -1
    return start

In [ ]:
app = dash.Dash(__name__)
app.layout = html.Div(style={'text-align': 'center'}, children=[
    html.H1('Flats for sel and rent in Warsaw',style={'textAlign': 'center','front-size' :50}),
    html.Br(),
    html.H2('red - sell',style={'textAlign': 'center', 'color': 'red', 'font-size': 30}),
    html.H2('blue - rent',style={'textAlign': 'center', 'color': 'blue', 'font-size': 30}),
    html.Br(),
    html.H2('Sell'),
    dcc.Input(id="Price_MIN_Sell", type="number", placeholder="Price MIN", value=400000),
    dcc.Input(id="Price_MAX_Sell", type="number", placeholder="Price MAX", value=450000),
    html.Button(id='my-button_Sell', n_clicks=0, children="Update"),
    html.Br(),
    html.H2('Rent'),
    dcc.Input(id="Price_MIN_Rent", type="number", placeholder="Price MIN", value=1000),
    dcc.Input(id="Price_MAX_Rent", type="number", placeholder="Price MAX", value=10000),
    html.Button(id='my-button_Rent', n_clicks=0, children="Update"),
    html.Br(),
    html.Iframe(id = 'map', srcDoc = world_new().get_root().render(),
                width = '700',height = '700')
    ])
@app.callback(
    dash.dependencies.Output('map', 'srcDoc'),
    [dash.dependencies.State('Price_MIN_Sell', 'value'),
     dash.dependencies.State('Price_MAX_Sell', 'value'),
     dash.dependencies.State('Price_MIN_Rent', 'value'),
     dash.dependencies.State('Price_MAX_Rent', 'value')],
    [dash.dependencies.Input('my-button_Sell', 'n_clicks'),
    dash.dependencies.Input('my-button_Rent', 'n_clicks')]
    )
def Rent_Price_Limiter(Price_MIN_Rent, Price_MAX_Rent, n_clicks):
    my_world = world_new()
    lok_rent_limit = lok_rent[lok_rent.price.between(Price_MIN_Rent, Price_MAX_Rent, inclusive=False)]
    lok_rent_limit = lok_rent_limit.reset_index(drop=True)
    for row in range(len(lok_rent.index)):
        folium.CircleMarker(
            location=[lok_rent.at[row,'latitude'], lok_rent.at[row,'longitude']],
            radius=3,
            popup='Rent price: '+ str(lok_rent.at[row,'rent_price']),
            color='blue',
            fill=True,
            fill_color='blue',
            fill_opacity=1
            ).add_to(my_world)

    lok_sell_limit = lok_sell[lok_sell.price.between(Price_MIN_Sell, Price_MAX_Sell, inclusive=False)]
    lok_sell_limit = lok_sell_limit.reset_index(drop=True)
    for row in range(len(lok_sell_limit.index)):
        folium.CircleMarker(
            location=[lok_sell_limit.at[row,'latitude'], lok_sell_limit.at[row,'longitude']],
            radius=3,
            popup='Sell price: ' + str(lok_sell_limit.at[row,'price']),
            color='red',
            fill=True,
            fill_color='red',
            fill_opacity=1
        ).add_to(my_world)
    html_string = my_world.get_root().render()
    return html_string

if __name__ == '__main__':
    app.run_server(host='0.0.0.0')

Dash is running on http://0.0.0.0:8050/

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on all addresses.
 * Running on http://172.17.0.2:8050/ (Press CTRL+C to quit)
192.168.10.24 - - [25/Sep/2021 20:43:54] "GET / HTTP/1.1" 200 -
192.168.10.24 - - [25/Sep/2021 20:43:54] "GET /_dash-dependencies HTTP/1.1" 200 -
192.168.10.24 - - [25/Sep/2021 20:43:54] "GET /_dash-layout HTTP/1.1" 200 -
192.168.10.24 - - [25/Sep/2021 20:43:54] "GET /_favicon.ico?v=2.0.0 HTTP/1.1" 200 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/opt/conda/lib/python3.9/site-packages/flask/app.py", line 2070, in wsgi_app
    response = self.full_dispatch_request()
  File "/opt/conda/lib/python3.9/site-packages/flask/app.py", line 1515, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/opt/conda/lib/python3.9/site-packages/flask/app.py", line 1513, in full_dispatch_request
    rv = self.dispatch_request()
  File "/opt/conda/lib/python3.9/site-packages/flask/app.py", line 1499, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**req.view_args)
  File "/opt/conda/lib/python3.9/site-packages/dash/dash.py", line 1336, in dispatch
    response.set_data(func(*args, outputs_list=outputs_list))
  File "/opt/conda/lib/python3.9/site-packages/dash/_callback.py", line 151, in add_context
    output_value = func(*func_args, **func_kwargs)  # %% callback invoked %%
TypeError: Rent_Price_Limiter() ta

192.168.10.24 - - [25/Sep/2021 20:43:54] "POST /_dash-update-component HTTP/1.1" 500 -
192.168.10.24 - - [25/Sep/2021 20:43:54] "GET /_favicon.ico?v=2.0.0 HTTP/1.1" 200 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/opt/conda/lib/python3.9/site-packages/flask/app.py", line 2070, in wsgi_app
    response = self.full_dispatch_request()
  File "/opt/conda/lib/python3.9/site-packages/flask/app.py", line 1515, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/opt/conda/lib/python3.9/site-packages/flask/app.py", line 1513, in full_dispatch_request
    rv = self.dispatch_request()
  File "/opt/conda/lib/python3.9/site-packages/flask/app.py", line 1499, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**req.view_args)
  File "/opt/conda/lib/python3.9/site-packages/dash/dash.py", line 1336, in dispatch
    response.set_data(func(*args, outputs_list=outputs_list))
  File "/opt/conda/lib/python3.9/site-packages/dash/_callback.py", line 151, in add_context
    output_value = func(*func_args, **func_kwargs)  # %% callback invoked %%
TypeError: Rent_Price_Limiter() ta

...

192.168.10.24 - - [25/Sep/2021 20:44:07] "POST /_dash-update-component HTTP/1.1" 500 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/opt/conda/lib/python3.9/site-packages/flask/app.py", line 2070, in wsgi_app
    response = self.full_dispatch_request()
  File "/opt/conda/lib/python3.9/site-packages/flask/app.py", line 1515, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/opt/conda/lib/python3.9/site-packages/flask/app.py", line 1513, in full_dispatch_request
    rv = self.dispatch_request()
  File "/opt/conda/lib/python3.9/site-packages/flask/app.py", line 1499, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**req.view_args)
  File "/opt/conda/lib/python3.9/site-packages/dash/dash.py", line 1336, in dispatch
    response.set_data(func(*args, outputs_list=outputs_list))
  File "/opt/conda/lib/python3.9/site-packages/dash/_callback.py", line 151, in add_context
    output_value = func(*func_args, **func_kwargs)  # %% callback invoked %%
TypeError: Rent_Price_Limiter() ta

192.168.10.24 - - [25/Sep/2021 20:44:20] "POST /_dash-update-component HTTP/1.1" 500 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/opt/conda/lib/python3.9/site-packages/flask/app.py", line 2070, in wsgi_app
    response = self.full_dispatch_request()
  File "/opt/conda/lib/python3.9/site-packages/flask/app.py", line 1515, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/opt/conda/lib/python3.9/site-packages/flask/app.py", line 1513, in full_dispatch_request
    rv = self.dispatch_request()
  File "/opt/conda/lib/python3.9/site-packages/flask/app.py", line 1499, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**req.view_args)
  File "/opt/conda/lib/python3.9/site-packages/dash/dash.py", line 1336, in dispatch
    response.set_data(func(*args, outputs_list=outputs_list))
  File "/opt/conda/lib/python3.9/site-packages/dash/_callback.py", line 151, in add_context
    output_value = func(*func_args, **func_kwargs)  # %% callback invoked %%
TypeError: Rent_Price_Limiter() ta

192.168.10.24 - - [25/Sep/2021 20:44:21] "POST /_dash-update-component HTTP/1.1" 500 -
192.168.10.24 - - [25/Sep/2021 20:44:23] "GET /_dash-component-suites/dash/dcc/dash_core_components-shared.js.map HTTP/1.1" 200 -
192.168.10.24 - - [25/Sep/2021 20:44:23] "GET /_dash-component-suites/dash/dash_table/bundle.js.map HTTP/1.1" 200 -
192.168.10.24 - - [25/Sep/2021 20:44:23] "GET /_dash-component-suites/dash/dcc/dash_core_components.js.map HTTP/1.1" 200 -
192.168.10.24 - - [25/Sep/2021 20:44:23] "GET /_dash-component-suites/dash/html/dash_html_components.min.js.map HTTP/1.1" 200 -
